# Explo design - comptage nombre de prélèvement par commune par catégorie
***
_Les parties indispensables à run sont indiquées avec la mention [TO RUN], afin de reproduire les analyses se situant sous ces sections._

**Contexte** :<br>
L'équipe Design a besoin d'aide des DAs !<br>
Pour choisir le meilleur mode d'affichage entre des points ou des polygones (coloration du tracé d'une commune sur la carte), nous avons besoin d'informations sur la volumétrie

**A faire** :<br>
Pour chaque catégorie retenue par Pauline, et pour chaque année : extraire le nbr de commune avec au moins un paramètre de la catégorie en question ayant fait l'objet d'un prélèvement

Catégories (</strong>utiliser le mapping fourni par Pauline)<br>
- Pesticide / Métabolites de pesticides
- Nitrite /Nitrate
- PFAS
- CVM
- 1,4 dioxane
- Perchlorate


Résultat : table avec ces colonnes:<br>
- année
- catégorie
- nombre de commune avec au moins un paramètre de la catégorie en question ayant fait l'objet d'un prélèvement


# Etude
***
### Packages + read data

In [1]:
# [TO RUN]  Packages
import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

In [3]:
# [TO RUN]
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [4]:
# [OPTIONAL] Tables dispo dans la bdd
con.sql("SHOW TABLES").show()

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ cog_communes            │
│ edc_communes            │
│ edc_prelevements        │
│ edc_resultats           │
│ laposte_communes        │
│ mapping_categories      │
│ stg_communes__cog       │
│ stg_communes__laposte   │
│ stg_edc__communes       │
│ stg_edc__prevelevements │
│ stg_edc__resultats      │
├─────────────────────────┤
│         11 rows         │
└─────────────────────────┘



Repartons du travail fait sur la tâche #155 Proposition création table agrégée commune et année avec résultat CVM 

- J'ajoute `mapping_categories AS CAT` car on s'intéresse aux catégories définies par Pauline 
- Pour RESULTS je prends tout `edc_resultats` pour la jointure avec `CAT` et car il n'y a pas besoin filtre (sur la substance)  ou calcul (de conformité)
- J'ai supprimé la table `COG` car on ne s'intéresse pas au nom des communes, `edc_communes` devrait suffire.

In [10]:
query_com_year_stat = """
WITH

UDI_COM AS (
    SELECT
      edc_communes.cdreseau,
      edc_communes.de_partition,
      edc_communes.inseecommune,
    FROM
      edc_communes
),
    
PVL AS (
	SELECT
	  referenceprel,
	  cdreseau,
      de_partition
	FROM   
      edc_prelevements
),

CAT AS (
    SELECT
      *
    FROM
      mapping_categories
)

    SELECT
      UDI_COM.de_partition AS annee,
      CAT.categorie AS categorie,
      COUNT(DISTINCT UDI_COM.inseecommune) AS nb_commune,
    FROM 
        UDI_COM
    LEFT JOIN
    	PVL 
    ON
    	UDI_COM.cdreseau = PVL.cdreseau
        AND UDI_COM.de_partition = PVL.de_partition
    LEFT JOIN 
        edc_resultats AS RESULTS
    ON
       RESULTS.referenceprel = PVL.referenceprel
       AND RESULTS.de_partition = PVL.de_partition
    LEFT JOIN
        CAT
    ON 
      RESULTS.cdparametre =   CAT.cdparametre
      AND RESULTS.libmajparametre = CAT.libmajparametre
      AND  RESULTS.cdparametresiseeaux = CAT.cdparametresiseeaux
      AND  RESULTS.casparam = CAT.casparam
    GROUP BY 
      	UDI_COM.de_partition,
        CAT.categorie
"""

df_com_year_stat = con.sql(query_com_year_stat).df()
df_com_year_stat

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,annee,categorie,nb_commune
0,2020,None,34788
1,2020,cvm,32304
2,2020,métaux lourds,32617
3,2020,sous produit désinfection,31026
4,2020,pesticides,29949
...,...,...,...
81,2023,pesticides,30494
82,2023,métaux lourds,32655
83,2024,non classé,32151
84,2024,phénol,28687


In [12]:
# pour pd.to_excel
!uv pip install openpyxl 

Using Python 3.12.9 environment at: C:\Users\Vinca\Documents\D4G_2025\13_pollution_eau\.venv
Audited 1 package in 892ms


In [13]:
# To excel
df_com_year_stat.to_excel("D4G_stat_com_per_year.xlsx")